#### Unset common proxy environment variables

In [2]:
import os

proxy_vars = [
    'http_proxy',
    'https_proxy',
    'ftp_proxy',
    'all_proxy',
    'HTTP_PROXY', 
    'HTTPS_PROXY',
    'FTP_PROXY',
    'ALL_PROXY',
    'SOCKS_PROXY',
    'socks_proxy'
]

for var in proxy_vars:
    if var in os.environ:
        del os.environ[var]

In [6]:
from langchain_core.documents import Document

In [3]:
# Create example documents about pets with metadata
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
documents

In [ ]:
type(documents[0])

## PDF Loader


In [2]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
file_path = "pdf/nke-10k-2023.pdf"

loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
docs[0].metadata

In [ ]:
print(docs[0].page_content)

## Text Splitter

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

## Embeddings

In [8]:
from langchain_ollama import OllamaEmbeddings

In [9]:
# 远程Ollama服务ip
import platform
REMOTE_HOST = "192.168.11.8" if platform.system() == "Linux" else "127.0.0.1"
# Ollama服务端口
OLLAMA_PORT = 11434
base_url=f"http://{REMOTE_HOST}:{OLLAMA_PORT}"
base_url

'http://127.0.0.1:11434'

In [10]:
embeddings = OllamaEmbeddings(model="deepseek-r1:8b", base_url=base_url)

In [11]:
embeddings

OllamaEmbeddings(model='deepseek-r1:8b', base_url='http://127.0.0.1:11434', client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

In [ ]:
len(vector_1)

## Vector stores

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

In [12]:
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)
len(ids)

In [ ]:
ids[:5]

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

In [ ]:
print(results[0].metadata.keys())

In [ ]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

In [ ]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

In [ ]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

## Example: My Resume

In [4]:
file_path_1 = "pdf/my_resume.pdf"

docs_1 = PyPDFLoader(file_path_1).load()

print(len(docs_1))

1


In [5]:
print(docs_1[0])

page_content='教育背景
中国农业大学 
985工程 本科 2011.09 - 2016.06
车 辆 工 程 汽车电子方向
湖北大学 硕士   2021.09 - 2024.06
计 算 机 技 术 人工智能方向
主修课程 高级软件工程、高级数据库、数据挖掘、数字图像处理、机器学习
研究课题
多智能体强化学习
强化学习
深度学习
自动驾驶
运动规划
科研成果 《Multi-Agent Reinforcement Learning with Social Learning for Autonomous Driving》 （SCI一区，审稿中）
《Motion Planning for Autonomous Driving: The State of the Art and Future Perspectives》 IEEE TIV，SCI一区，影响因子8.2
工作经历
成都光恒互动科技有限公司 游戏AI算法工程师
实习 2024.01 - 2024.03
工作内容 负责棋类游戏AI算法 、 的游戏NPC 等算法的设计 、 研究与开发
相关算法： 等
相关技术：C++ 、 Python 、 Docker 、 UE5等。
NLP驱动 行为决策
深度 学习 （有监督 、 CNN） 、 强化 学习 （PPO/DQN）
中科 慧拓 （北 京 ）科技有限公司 科研 助理 （ 矿 车自动驾驶）
实习 2022.09 - 2022.12
工作内容 负 责 协 助 智 慧 矿 山 矿 车 的 产 品 研 发 ， 以 及 基 于 的 多 车 协 同 算 法 研 究 ， 包 括 数
据 清 理 和标注，模拟 器 /环境搭建，论文跟进与复现等。
自 动 驾 驶 多 智 能 体 强 化 学 习
宜昌八达 工程技术 咨询 有限公司 设 计 师
全职 201 7  - 2020
工作内容 负 责 公 路 、 桥 梁 工 程 的 勘 测 、 设 计 与 造 价 。 能 够 独 立 完 成 设 计 任 务 ， 熟 练 使 用 地 质 勘 测 工 具 ， 累 计 设 计
里 程 超 千 公 里 ， 包 括 国 省 道 、 5A 级 景 区 旅 游 公 路 等 。 期 间 还 根 据 本 行 业 特 点 和 业 务 流 程 ， 自 学 VB编 程
语言 ， 了一套 基于 VB A的E x c

In [13]:
ids = vector_store.add_documents(documents=docs_1)
len(ids)

1

In [14]:
results = vector_store.similarity_search_with_score("US")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.03750337037736621

page_content='教育背景
中国农业大学 
985工程 本科 2011.09 - 2016.06
车 辆 工 程 汽车电子方向
湖北大学 硕士   2021.09 - 2024.06
计 算 机 技 术 人工智能方向
主修课程 高级软件工程、高级数据库、数据挖掘、数字图像处理、机器学习
研究课题
多智能体强化学习
强化学习
深度学习
自动驾驶
运动规划
科研成果 《Multi-Agent Reinforcement Learning with Social Learning for Autonomous Driving》 （SCI一区，审稿中）
《Motion Planning for Autonomous Driving: The State of the Art and Future Perspectives》 IEEE TIV，SCI一区，影响因子8.2
工作经历
成都光恒互动科技有限公司 游戏AI算法工程师
实习 2024.01 - 2024.03
工作内容 负责棋类游戏AI算法 、 的游戏NPC 等算法的设计 、 研究与开发
相关算法： 等
相关技术：C++ 、 Python 、 Docker 、 UE5等。
NLP驱动 行为决策
深度 学习 （有监督 、 CNN） 、 强化 学习 （PPO/DQN）
中科 慧拓 （北 京 ）科技有限公司 科研 助理 （ 矿 车自动驾驶）
实习 2022.09 - 2022.12
工作内容 负 责 协 助 智 慧 矿 山 矿 车 的 产 品 研 发 ， 以 及 基 于 的 多 车 协 同 算 法 研 究 ， 包 括 数
据 清 理 和标注，模拟 器 /环境搭建，论文跟进与复现等。
自 动 驾 驶 多 智 能 体 强 化 学 习
宜昌八达 工程技术 咨询 有限公司 设 计 师
全职 201 7  - 2020
工作内容 负 责 公 路 、 桥 梁 工 程 的 勘 测 、 设 计 与 造 价 。 能 够 独 立 完 成 设 计 任 务 ， 熟 练 使 用 地 质 勘 测 工 具 ， 累 计 设 计
里 程 超 千 公 里 ， 包 括 国 省 道 、 5A 级 景 区 旅 游 公 路 等 。 期 间 还 根 据 本 行 业 特 点 和 业 务 流 程 ， 自 学 

In [17]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

In [18]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    # web_paths=("https://www.cnblogs.com/Chilam007/p/18692699",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
len(docs)

1

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

66

In [20]:
_ = vector_store.add_documents(documents=all_splits)